In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# ========== python ==========
import os
from pathlib import Path
from logging import Logger
from typing import List, Dict, Tuple, Optional, Union, Callable, Final, Literal, get_args
from operator import itemgetter, attrgetter
import itertools
from IPython.display import display

from utils.setup import setup_logger, get_device
from const.const_values import PROJECT_DIR

os.chdir(PROJECT_DIR)
logger: Logger = setup_logger(__name__, f'{PROJECT_DIR}/log/jupyter_run.log')
device = get_device(device_name='cpu', logger=logger)

In [2]:
# jupyter
import seaborn as sns
import matplotlib.pyplot as plt
# Machine learning
import numpy as np
import pandas as pd
import h5py
import optuna
# torch
import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from torch.utils.tensorboard.writer import SummaryWriter
# torch ignite
from ignite.engine import Engine
from ignite.handlers import Checkpoint
# My items
from models.datasets.data_helper import MyDataHelperForStory, MyDataLoaderHelper, DefaultTokens
from models.datasets.datasets_for_story import StoryTriple
# My utils
from utils.setup import load_param
from utils.torch import load_model, torch_fix_seed
# main function
from run_for_KGC import main_function


In [3]:
from const.const_values import CPU, MODEL
from models.KGModel.kg_model import HEAD, RELATION, TAIL
from utils.torch_ignite import TRAINER, EVALUATOR
from const.const_values import DATASETS, DATA_HELPER, DATA_LOADERS, TRAIN_RETURNS

In [4]:
SEED: Final[int] = 42
args_path = f'{PROJECT_DIR}/models/kgc03/ResidentPatient090/param.pkl'
model_path = f'{PROJECT_DIR}/models/kgc03/ResidentPatient090/model.pth'

In [5]:
args = load_param(args_path)

# args.pre_train = True
args.logger = logger
args.device = device
args.batch_size = 1
args.pre_train=False
args.init_embedding_using_bert = False
args.model_path = model_path
args.only_load_trainer_evaluator = True

del args.optuna_file, args.device_name, args.pid, args.study_name, args.n_trials
logger.info(args)

2023-02-04 06:49:07 - INFO - 2517384385.py - 13 - Namespace(notebook=False, console_level='info', logfile='models/kgc03/ResidentPatient090/log.log', param_file='models/kgc03/ResidentPatient090/param.pkl', train_anyway=False, old_data=0, tensorboard_dir='models/kgc03/ResidentPatient090/tensorboard', checkpoint_dir='models/kgc03/ResidentPatient090/checkpoint/', model_path='/Users/ryoyakaneda/Documents/学校/M1Study/knowledge_graph/models/kgc03/ResidentPatient090/model.pth', resume_from_checkpoint=False, resume_from_last_point=False, only_load_trainer_evaluator=True, resume_checkpoint_path=None, pre_train=False, train_valid_test=False, only_train=True, use_for_challenge100=False, use_for_challenge090=True, use_for_challenge075=False, use_title='ResidentPatient', do_optuna=False, story_special_num=5, relation_special_num=5, entity_special_num=5, padding_token_e=0, cls_token_e=1, mask_token_e=2, sep_token_e=3, bos_token_e=4, padding_token_r=0, cls_token_r=1, mask_token_r=2, sep_token_r=3, bos_

In [6]:
torch_fix_seed(seed=SEED)
return_dict = main_function(args, logger=logger)

2023-02-04 06:49:07 - INFO - run_for_KGC.py - 893 - ----- make datahelper start. -----
2023-02-04 06:49:07 - INFO - data_helper.py - 333 - ====== Show MyRawData ======
2023-02-04 06:49:07 - INFO - data_helper.py - 334 - entity num: 7812
2023-02-04 06:49:07 - INFO - data_helper.py - 335 - relation num: 62
2023-02-04 06:49:07 - INFO - data_helper.py - 336 - ====== Show MyRawData ======
2023-02-04 06:49:07 - INFO - data_helper.py - 607 - ========== Show DataHelper ==========
2023-02-04 06:49:07 - INFO - data_helper.py - 333 - ====== Show MyRawData ======
2023-02-04 06:49:07 - INFO - data_helper.py - 334 - entity num: 7812
2023-02-04 06:49:08 - INFO - data_helper.py - 335 - relation num: 62
2023-02-04 06:49:08 - INFO - data_helper.py - 336 - ====== Show MyRawData ======
2023-02-04 06:49:08 - INFO - data_helper.py - 609 - entity_special_dicts: {0: '<pad_e>', 1: '<cls_e>', 2: '<mask_e>', 3: '<sep_e>', 4: '<bos_e>'}
2023-02-04 06:49:08 - INFO - data_helper.py - 610 - relation_special_dicts: {

In [7]:
model = return_dict[MODEL]

dataset_train: StoryTriple = return_dict[DATASETS][0]
triple: torch.Tensor = dataset_train.triple
data_helper: MyDataHelperForStory = return_dict[DATA_HELPER]
evaluator: Checkpoint = return_dict[TRAIN_RETURNS][EVALUATOR]

load_model(model, args.model_path, device)
model.eval()

entities, relations = data_helper.processed_entities, data_helper.processed_relations
d_e, d_r = {e: i for i, e in enumerate(entities)}, {r: i for i, r in enumerate(relations)}

triple_df = pd.DataFrame([(entities[_t[0]], relations[_t[1]], entities[_t[2]]) for _t in triple], columns=[HEAD, RELATION, TAIL])
story_entities = triple_df[HEAD].tolist()

In [8]:
triple_df

,head,relation,tail
0,<bos_e>,<bos_r>,<bos_e>
1,AbbeyGrange:001,kgc:subject,AllTitle:Holmes
2,AbbeyGrange:001,kgc:hasPredicate,word.predicate:wakeUp
3,AbbeyGrange:001,kgc:whom,AllTitle:Watson
4,AbbeyGrange:001,kgc:infoSource,AllTitle:Watson
...,...,...,...
17712,AbbeyGrange:084,kgc:subject,AbbeyGrange:Theresa
17713,AbbeyGrange:084,kgc:hasPredicate,word.predicate:rest
17714,AbbeyGrange:084,kgc:where,AbbeyGrange:room_of_Lady_Brackenstall
17715,<bos_e>,<bos_r>,<bos_e>


In [9]:
def extract(_model, target, inputs):
    features = None

    def forward_hook(_module, _inputs, _):
        nonlocal features
        x, _, _ = _inputs
        outputs = _module.forward(x, x, x, need_weights=True)[1]
        features = outputs.detach().clone()

    handle = target.register_forward_hook(forward_hook)

    _model.eval()
    _model(inputs, torch.LongTensor([[]]), torch.LongTensor([[]]), torch.LongTensor([[]]))

    handle.remove()

    return features

def get_attention(input_):
    assert len(input_) == 1
    features = extract(model, model.transformer.layers[-1].self_attn, input_)[0]
    df_attention = pd.DataFrame([[entities[h], relations[r], entities[t]]+[features[j, i].item() for j in range(len(features))] for i, (h, r, t) in enumerate(input_[0])])
    df_attention.columns=[HEAD, RELATION, TAIL] + [f'atten_from{i}' for i in range(len(df_attention.columns)-3)]
    return df_attention

def show_attension_heatmap(df_attention):
    sns.heatmap(df_attention.iloc[:,3:])
    plt.show()

In [10]:
MASK_E = DefaultTokens.MASK_E
KILL = 'word.predicate:kill'

TAKE = 'word.predicate:take'
BRING = 'word.predicate:bring'
DIE = 'word.predicate:die'
HIDE = 'word.predicate:hide'

In [11]:
bos_triple = [d_e[DefaultTokens.BOS_E], d_r[DefaultTokens.BOS_R],d_e[DefaultTokens.BOS_E]]
mask_e_id = d_e[DefaultTokens.MASK_E]
Holmes_id =d_e['AllTitle:Holmes']

def make_ranking(from_story_name, to_story_name, predicate_, whom_, subject_, why_, what_, where_):
    if not (from_story_name is None and to_story_name is None):
        _start_index = story_entities.index(from_story_name)-1 
        _end_index = len(story_entities) - story_entities[::-1].index(to_story_name)
    else: 
        _start_index = 0
        _end_index = 0
    question_ = torch.tensor(
        [
            bos_triple, 
            [mask_e_id, d_r['kgc:infoSource'],     Holmes_id      ],
            [mask_e_id, d_r['kgc:hasPredicate'],   d_e[predicate_]],
            [mask_e_id, d_r['kgc:whom'],           d_e[whom_     ]],
            [mask_e_id, d_r['kgc:subject'],        d_e[subject_  ]],
            [mask_e_id, d_r['kgc:why'],            d_e[why_      ]],
            [mask_e_id, d_r['kgc:what'],           d_e[what_     ]],
            [mask_e_id, d_r['kgc:where'],          d_e[where_    ]],
        ]
    )
    mask_ = torch.zeros_like(question_, dtype=torch.bool) # not mask all position
    mask_[1:, 0] = True                                   # where head position without bos token
    mask_[1:, 2] = True                                   # where tail position without bos token

    last_triples = triple[_start_index: _end_index]

    questions = torch.cat([last_triples, question_], dim=0).unsqueeze(0)
    masks = torch.cat([torch.zeros_like(last_triples), mask_], dim=0).to(torch.bool).transpose(1,0).unsqueeze(0)

    data_list = []
    with torch.no_grad():
        _, (story_pred, relation_pred, entity_pred) = model(questions, masks[:,0], masks[:,1], masks[:,2])
        sorted_ = torch.argsort(entity_pred, dim=1, descending=True)
        for i in range(sorted_.shape[1]):
            ans_= sorted_[:, i]
            info_source_, predicate_pred, whom_pred, subject_pred, why_pred, what_pred, where_pred = ans_
            data_list.append([entities[predicate_pred], entities[whom_pred], entities[subject_pred], entities[why_pred], entities[what_pred], entities[where_pred]])
    df_ranking = pd.DataFrame(data_list, columns=['predicate', 'whom', 'subject', 'why', 'what', 'where'])
    df_attension = get_attention(questions)
    
    return df_ranking, df_attension

In [12]:
def main_func01(_title, _victim_name, criminal, predicate, _last_index, _story_len):
    from_ = f'{_title}:{_last_index-_story_len+1}'
    to_ = f'{_title}:{_last_index}'
    predicate = predicate
    victim = f'{_title}:{_victim_name}'
    criminal = f'{_title}:{criminal}'
    df_ranking, df_attension = make_ranking(
        from_, to_, predicate, victim, MASK_E, MASK_E, MASK_E, MASK_E)

    pred_rank = df_ranking.index[df_ranking['subject']==criminal].tolist()
    pred_rank = pred_rank[0] if len(pred_rank)==1 else -1
    logger.info(f"The pred ranking about {criminal} is {pred_rank}")
    display(df_ranking.iloc[:max(20, pred_rank)])
    len_ = len(df_attension)
    for i in range(len_-10, len_):
        print(f"index={i}, triple={df_attension.iloc[i,:3].tolist()}, attention list")
        display(df_attension.sort_values(f'atten_from{i}', ascending=False).iloc[:,[0,1,2,3+i]],)
    return df_ranking, df_attension

def check_killer(_title, _victim_name, _killer_name, _last_index, _story_len):
    return main_func01(_title, _victim_name, _killer_name, KILL, _last_index, _story_len)

In [13]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

### 入院患者
Who killed Blessington? (criminal & explanation)
被害者: Blessington
犯人: 3人
犯行動機:

In [14]:
def do_ResidentPatient_pred():
    title = 'ResidentPatient'
    victim_name = 'Blessington'
    killer_name = ''
    last_index = 291
    story_len = 80

    df_ranking, df_attention = check_killer(title, victim_name, killer_name, last_index, story_len)
    return df_ranking, df_attention

do_ResidentPatient_pred()

2023-02-04 06:49:08 - INFO - 1707532137.py - 12 - The pred ranking about ResidentPatient: is -1


,predicate,whom,subject,why,what,where
0,word.predicate:enter,ResidentPatient:Blessington,ResidentPatient:Young_man,word.predicate:see,SpeckledBand:1883-04-01T16,ResidentPatient:Blessington_room
1,word.predicate:meet,ResidentPatient:Young_man,ResidentPatient:Elderly_man,SpeckledBand:Helen_s_bedroom,ResidentPatient:Blessington,SpeckledBand:Julia_s_bedroom
2,word.predicate:say,ResidentPatient:Elderly_man,ResidentPatient:Helper_boy,SpeckledBand:Helen,SilverBlaze:Silver_Blaze,<bos_e>
3,word.predicate:notHave,AllTitle:Holmes,AllTitle:Holmes,word.predicate:notHave,SpeckledBand:door,word.predicate:see
4,word.predicate:exist,ResidentPatient:Helper_boy,ResidentPatient:Blessington,DevilsFoot:window,ResidentPatient:Elderly_man,ACaseOfIdentity:Windibank
5,word.predicate:die,word.predicate:go,AbbeyGrange:Thought_of_Holmes,SpeckledBand:Julia,<bos_e>,SpeckledBand:1883-04-01T15
6,word.predicate:want,ResidentPatient:Percy_Trevelyan,DevilsFoot:Porter,DateTime:1881-04-02T09:00:00,word.predicate:arrive,SpeckledBand:1883-04-01T14
7,word.predicate:think,SilverBlaze:Silver_Blaze,SilverBlaze:Silver_Blaze,SilverBlaze:stable,word.predicate:go,word.predicate:guess
8,word.predicate:arrive,CrookedMan:Henry,AllTitle:Watson,CrookedMan:1887-07-06T21:15:00,DancingMen:Abe_Slaney,SilverBlaze:police
9,AllTitle:Holmes,AbbeyGrange:Thought_of_Holmes,word.predicate:go,SpeckledBand:Julia_s_bedroom,AbbeyGrange:Jack_Croker,word.predicate:go


index=407, triple=['ResidentPatient:291', 'kgc:subject', 'ResidentPatient:Helper_boy'], attention list


,head,relation,tail,atten_from407
216,ResidentPatient:255,kgc:time,DateTime:1889-10-21T05:30:00,0.021525
215,ResidentPatient:255,kgc:when,ResidentPatient:1889-10-21T05:30:00,0.012313
339,ResidentPatient:277,kgc:subject,ResidentPatient:Young_man,0.010106
378,ResidentPatient:285,kgc:subject,ResidentPatient:Young_man,0.009347
214,ResidentPatient:255,kgc:when,ResidentPatient:After_exiting,0.008958
182,ResidentPatient:248,kgc:subject,ResidentPatient:Someone,0.008106
248,ResidentPatient:262,kgc:what,ResidentPatient:266,0.007924
379,ResidentPatient:285,kgc:subject,ResidentPatient:Helper_boy,0.007791
338,ResidentPatient:277,kgc:subject,ResidentPatient:Elderly_man,0.007349
285,ResidentPatient:270,kgc:hasPredicate,word.predicate:cannotFind,0.007069


index=408, triple=['ResidentPatient:291', 'kgc:hasPredicate', 'word.predicate:wander'], attention list


...

index=415, triple=['<mask_e>', 'kgc:what', '<mask_e>'], attention list


,head,relation,tail,atten_from415
216,ResidentPatient:255,kgc:time,DateTime:1889-10-21T05:30:00,0.016254
248,ResidentPatient:262,kgc:what,ResidentPatient:266,0.011265
410,<mask_e>,kgc:infoSource,AllTitle:Holmes,0.010962
184,ResidentPatient:248,kgc:what,ResidentPatient:Another_two_butts_of_Cigar,0.010129
182,ResidentPatient:248,kgc:subject,ResidentPatient:Someone,0.009226
247,ResidentPatient:262,kgc:what,ResidentPatient:265,0.009117
368,ResidentPatient:282,kgc:infoSource,AllTitle:Holmes,0.009069
249,ResidentPatient:262,kgc:what,ResidentPatient:267,0.008877
243,ResidentPatient:262,kgc:subject,AllTitle:Holmes,0.008283
297,ResidentPatient:271,kgc:what,ResidentPatient:279,0.007974


index=416, triple=['<mask_e>', 'kgc:where', '<mask_e>'], attention list


,head,relation,tail,atten_from416
216,ResidentPatient:255,kgc:time,DateTime:1889-10-21T05:30:00,0.014203
215,ResidentPatient:255,kgc:when,ResidentPatient:1889-10-21T05:30:00,0.011436
182,ResidentPatient:248,kgc:subject,ResidentPatient:Someone,0.010229
243,ResidentPatient:262,kgc:subject,AllTitle:Holmes,0.010222
248,ResidentPatient:262,kgc:what,ResidentPatient:266,0.009544
410,<mask_e>,kgc:infoSource,AllTitle:Holmes,0.009171
214,ResidentPatient:255,kgc:when,ResidentPatient:After_exiting,0.008771
186,ResidentPatient:249,kgc:subject,ResidentPatient:Someone,0.007907
387,ResidentPatient:286,kgc:where,ResidentPatient:Blessington_room,0.007762
361,ResidentPatient:281,kgc:to,ResidentPatient:Blessington_room,0.007746


(                       predicate                         whom  \
 0           word.predicate:enter  ResidentPatient:Blessington   
 1            word.predicate:meet    ResidentPatient:Young_man   
 2             word.predicate:say  ResidentPatient:Elderly_man   
 3         word.predicate:notHave              AllTitle:Holmes   
 4           word.predicate:exist   ResidentPatient:Helper_boy   
 ...                          ...                          ...   
 7812        word.predicate:ideal               CrookedMan:063   
 7813             SilverBlaze:119          ACaseOfIdentity:558   
 7814  AbbeyGrange:French_windows              DevilsFoot:Room   
 7815             AbbeyGrange:390          word.predicate:Have   
 7816            SpeckledBand:321   AbbeyGrange:French_windows   
 
                           subject                               why  \
 0       ResidentPatient:Young_man                word.predicate:see   
 1     ResidentPatient:Elderly_man      SpeckledBand:Helen_s_b